In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mat73
import time

# import jax
# import jax.numpy as jnp
# from jax import random, jit
import matplotlib.pyplot as plt

# ------------------     MATLAB STUFF  ----------------------------------
def get_matlab_variables(mat_file_path):
    variables = mat73.loadmat(mat_file_path)
    #double gets converted to np array by default
    value_func_data = variables['Vx']
    lx_data = variables['lx'] 
    tau2 = variables['tau2']

    #Deriv is cell which gets converted into list of lists
    deriv_x_data = np.array(variables['Deriv'][0])
    deriv_x_data = deriv_x_data.squeeze()
    deriv_y_data = np.array(variables['Deriv'][1])
    deriv_y_data = deriv_y_data.squeeze()
    deriv_th_data = np.array(variables['Deriv'][2])
    deriv_th_data = deriv_th_data.squeeze()
    
    #uOpt is also cell which gets converted into list of lists
    uOpt_vel = np.array(variables['uOpt'][0])
    uOpt_vel = uOpt_vel.squeeze()
    uOpt_angle = np.array(variables['uOpt'][1])
    uOpt_angle = uOpt_angle.squeeze() 

    #g is struct whic gets converted into dic
    #vs is cell which give a list
    x_coord=np.array(variables['g']['vs'][0])
    y_coord=np.array(variables['g']['vs'][1])
    th_coord=np.array(variables['g']['vs'][2])
    x_coord = x_coord.squeeze()
    y_coord = y_coord.squeeze()
    th_coord = th_coord.squeeze()

    matlab_var_dict = dict( value_func_data=value_func_data,
                            lx_data=lx_data,
                            deriv_x_data=deriv_x_data,
                            deriv_y_data=deriv_y_data,
                            deriv_th_data=deriv_th_data,
                            uOpt_vel=uOpt_vel,
                            uOpt_angle=uOpt_angle,
                            x_coord=x_coord,
                            y_coord=y_coord,
                            th_coord=th_coord,
                            tau2=tau2
                           )
    return matlab_var_dict


#---------------------- Load MATLAB ---------------------------------------------------------
#v3 added uopt lookup table
matlab_var_dict= get_matlab_variables('/home/javier/jax_work/mppi/rc_car_mppi/brt_rc_wh_fine_v3.mat')

data = matlab_var_dict['value_func_data']
data_lx = matlab_var_dict['lx_data']
uOpt_vel = matlab_var_dict['uOpt_vel']
uOpt_angle = matlab_var_dict['uOpt_angle']
coords = [matlab_var_dict['x_coord'], matlab_var_dict['y_coord'], matlab_var_dict['th_coord']]

data = np.array(data)
data_lx = np.array(data_lx)
uOpt_vel = np.array(uOpt_vel)
uOpt_angle = np.array(uOpt_angle)
coords = [np.array(coord) for coord in coords]



In [24]:

#open the npz file in the experiment folder
exp_path = '/home/javier/jax_work/mppi/rc_car_mppi/experiments/mppi_data_20240824-142443.npz'

#20240824-141345 full method
#20240824-142443

#20240824-141858 mppi-obs + LR 

npzfile = np.load(exp_path, allow_pickle=True)

state_history = npzfile['state_history']
control_history = npzfile['control_history']
hallucination_history = npzfile['hallucination_history']
hallucination_time_history = npzfile['hallucination_time_history']
loop_time_history = npzfile['loop_time_history']
lr_active = npzfile['lr_active']
m_t = [[0.8, 0.4, 0.4] if lr_active[i] == 1 else [0.4, 0.4, 0.8] for i in range(len(lr_active))]
experiment_params = npzfile['experiment_params']
experiment_params = experiment_params.item()

print(state_history.shape)
print(control_history.shape)
print(hallucination_history.shape)
print(experiment_params)

#remove first 5 timings
hallucination_time_history = hallucination_time_history[5:]
loop_time_history = loop_time_history[5:]

#get avg and std of the hallucination time and loop time
print(f"Average time for hallucimations {np.mean(hallucination_time_history):.1f} ± {np.std(hallucination_time_history):.1f} ms")
print(f"Average time for control loop  {np.mean(loop_time_history):.1f} ± {np.std(loop_time_history):.1f} ms")

# calculate cost of the trajectory

# @jit
# def cost_function(states, controls, data_lx, data, v_target=V_MAX):
#     x = states[0, :]
#     y = states[1, :]
#     th = states[2, :]
#     x_idx = jnp.argmin(jnp.abs(coords[0] - x[:,jnp.newaxis]),axis=1)
#     y_idx = jnp.argmin(jnp.abs(coords[1] - y[:,jnp.newaxis]),axis=1)
#     th_idx= jnp.argmin(jnp.abs(coords[2] - th[:,jnp.newaxis]),axis=1)
#     lx = data_lx[x_idx, y_idx]  # distance to nearest obstacle (max~0.45)
#     vx = data[x_idx, y_idx, th_idx]  # value function (max~1.69)
    
#     v = controls[:, 0]
#     #delta = controls[:, 1]
#     cost = jnp.sum((v - v_target) ** 2) * W_VEL  # magnitude 1.69
#     cost += jnp.sum(0.45-lx) * W_CENTERING # magnitude 0.45 * W
#     cost += jnp.sum(jnp.where(lx < 0.0, 1.0, 0.0)) * W_COLLISION  # magnitude 1 * W
#     cost += jnp.sum(jnp.where(vx < 0.0, 1.0, 0.0)) * W_IN_BRT  # magnitude 1 * W
#     return cost 


x = state_history[0, :]
y = state_history[1, :]
th = state_history[2, :]
x_idx = np.argmin(np.abs(coords[0] - x[:,np.newaxis]),axis=1)
y_idx = np.argmin(np.abs(coords[1] - y[:,np.newaxis]),axis=1)
th_idx= np.argmin(np.abs(coords[2] - th[:,np.newaxis]),axis=1)
lx = data_lx[x_idx, y_idx]  # distance to nearest obstacle (max~0.45)
vx = data[x_idx, y_idx, th_idx]  # value function (max~1.69)

v = control_history[:, 0]
#delta = controls[:, 1]
cost = np.sum((v - 0.8) ** 2) * experiment_params['W_VEL']  # magnitude 1.69
cost += np.sum(0.45-lx) * experiment_params['W_CENTERING'] # magnitude 0.45 * W
cost += np.sum(np.where(lx < 0.0, 1.0, 0.0)) * experiment_params['W_COLLISION']  # magnitude 1 * W
cost += np.sum(np.where(vx < 0.0, 1.0, 0.0)) * experiment_params['W_IN_BRT']  # magnitude 1 * W

print(f"Cost of the trajectory: {cost:.2f}")


# # Print the extracted range
# range_init = 850
# range_ends = 890
# print('lr_active:', lr_active[range_init:range_ends])


# range_test_point= 885
# print('hallucinations:\n', hallucination_history[range_test_point,100,:,0:6])

(1249, 3)
(1249, 2)
(1249, 1000, 3, 100)
{'DT': 0.02, 'L': 0.235, 'V_MIN': 0.7, 'V_MAX': 1.4, 'DELTA_MIN': -0.436, 'DELTA_MAX': 0.436, 'HALLUCINATION_STEPS': 100, 'NUM_THREADS': 1000, 'EXPERIMENT_THRESHOLD': 0.1, 'HALLUCINATIONS_THRESHOLD': 0.1, 'W_VEL': 1.0, 'W_CENTERING': 120.0, 'W_COLLISION': 50.0, 'W_IN_BRT': 0.0, 'TEMPERATURE': 0.005}
Average time for hallucimations 4.0 ± 1.7 ms
Average time for control loop  14.3 ± 4.7 ms
Cost of the trajectory: 455.03


In [25]:
from ipywidgets import interact

# Assuming state_history , matlab_var_dict, data_lx, data, m_t, and list_hallucinations_at_idx are defined elsewhere in your code
state_history = np.array(state_history)
control_history = np.array(control_history)

def plot_func(idx_to_plot):
    state_plot = state_history [idx_to_plot]
    # Print the state with 2 decimals
    print('[x,y,th]')
    print(np.around(state_plot, decimals=2))
    # Get the index of the closest v and th to the state_first_fltr
    th_idx = np.argmin(np.abs(matlab_var_dict['th_coord'] - state_plot[2]))

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(6, 18))

    ####race track#####
    data1 = data_lx
    data1 = data1.transpose()
    CP1 = ax1.contour(matlab_var_dict['x_coord'], matlab_var_dict['y_coord'], data1, 0, colors='red', linewidths=1)

    data2 = data[:, :, th_idx]  # Slice of lx
    data2 = data2.transpose()
    CP2 = ax1.contour(matlab_var_dict['x_coord'], matlab_var_dict['y_coord'], data2, 0, colors='blue', linewidths=1)

    # Grab closest theta
    th = matlab_var_dict['th_coord']
    th = th[th_idx]
    # Plot trajectory and final state
    SP1 = ax1.scatter(state_history [0:idx_to_plot, 0], state_history[0:idx_to_plot, 1], c=m_t[0:idx_to_plot], s=5, alpha=0.5) #
    SP2 = ax1.scatter(state_history [idx_to_plot, 0], state_history [idx_to_plot, 1], s=10, c=[[0.0, 0.0, 0.0]], alpha=1.0)
    # Plot arrow at final state
    ax1.arrow(state_history [idx_to_plot, 0], state_history [idx_to_plot, 1], np.cos(th) * 0.2, np.sin(th) * 0.2, head_width=0.1, head_length=0.1, fc='k', ec='k')

    # Plot the hallucinations corresponding to idx_to_plot        
    for i in range(0, len(hallucination_history[idx_to_plot]), 20):
        h = hallucination_history[idx_to_plot][i]
        ax1.plot(h[0, :], h[1, :], color='green', alpha=0.2)

    # Set x and y limits
    ax1.set_xlim([0, 6])
    ax1.set_ylim([0, 4])

    # Change aspect ratio to match the grid
    ax1.set_aspect('equal')

    ####velocity plot####
    ax2.plot(control_history[:idx_to_plot+1, 0])
    ax2.scatter(idx_to_plot, control_history[idx_to_plot, 0], s=10, c=[[0.0, 0.0, 0.0]], alpha=1.0)
    ax2.set_title('velocity control')
    ax2.set_xlabel('k')
    ax2.set_aspect(aspect=50)

    ####angle plot####
    ax3.plot(control_history[:idx_to_plot+1, 1])
    ax3.scatter(idx_to_plot, control_history[idx_to_plot, 1], s=10, c=[[0.0, 0.0, 0.0]], alpha=1.0)
    ax3.set_title('angle control')
    ax3.set_xlabel('k')
    ax3.set_aspect(aspect=50)
       
    
    plt.show()

# Use interact to create a slider for idx_to_plot
interact(plot_func, idx_to_plot=(0, len(state_history ) - 2, 1))

interactive(children=(IntSlider(value=623, description='idx_to_plot', max=1247), Output()), _dom_classes=('wid…

<function __main__.plot_func(idx_to_plot)>